In [1]:
#NOTE:
"uriyas model total emotions: happy, sad, disgust, fear, neutral, anger, surprise"
"use for valence: happy/sad"
"if needed, he'll use for arousal: neutral/suprise"
#THEN, for me:
#map the emotions to (x,y) coordinates of valence and arousal, then multiply it by the probability of the emotion, then take the weighted average and get the reslting x,y coordinates
#valence and arousal are the two dimensions of the circumplex model and the resulting scores should get put in one of the 9(nuetral plus 8 categories) categories

"if needed, he'll use for arousal: neutral/suprise"

# GoEmotions: Testing Input to the Model, Adding Softmax Layer, Conversion Function for final Label

## 1. Loading Model

In [7]:
import transformers
import tensorflow as tf

c:\Users\mlar5\anaconda3\envs\CudaTest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [98]:
#load the model monologg/bert-base-cased-goemotions-original from huggingface
model = transformers.AutoModelForSequenceClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")
tokenizer = transformers.AutoTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [9]:
#test the model on a sentence
sentence = "I am so happy"
tokens = tokenizer.encode(sentence, return_tensors="pt")
tokens = tokens.cuda()
result = model(tokens)

## 2. Playing around/testing model (skip)
this is to figure out what will be needed making softmax, and determining the classes

In [101]:
#determine the max size of the input the model can handle
max_length = model.config.max_position_embeddings
print(max_length)

512


In [10]:
#identify the top 3 emotion with the highest probability
emotion = result.logits
emotion = emotion.cpu().detach().numpy()
emotion = emotion[0]
emotion = emotion.argsort()[-3:][::-1]
emotion = emotion.tolist()
print(emotion)

[17, 0, 13]


In [11]:
#convert model.config.id2label to a list of emotions, where the key is the index of the emotion
emotion_dict = model.config.id2label

In [26]:
#identify the label of top 3 emotions from emotion list
for i in emotion:
    print(emotion_dict[i])


joy
admiration
excitement


In [8]:
#identify the emotion with the highest probability
emotion = result.logits.argmax()
print(emotion)
#which emotion is that?
print(model.config.id2label[emotion.item()])

tensor(17, device='cuda:0')
joy


In [28]:
emotion_dict.values()
emotionGroups=[]

dict_values(['admiration', 'amusement', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'anger', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment'])

In [30]:
emotionSet = set(emotion_dict.values())

In [31]:
approvedMoods=['neutral']

## 3. Softmax

### Function

In [71]:
def getSoftmax(model, sentence, n,printRawScores=False, printTopN=True):
    tokens = tokenizer.encode(sentence, return_tensors="pt")
    tokens = tokens.cuda()
    result = model(tokens)
    emotion = result.logits
    emotion = emotion.cpu().detach().numpy()
    emotion = emotion[0]
    softmax = tf.nn.softmax(emotion)
    if printRawScores:
        print(softmax)
    
    if printTopN:
        emotion = emotion.argsort()[-n:][::-1]
        emotion = emotion.tolist()
        printTopEmotions(emotion,model, softmax)
    return softmax


def printTopEmotions(emotion, model, softmax):
    
    #identify the label of top n emotions from emotion list
    #softmax is in the order of the values in emotion_dict so we can use emotion[id] to get the softmax value
    id=0
    emotion_dict = model.config.id2label
    for i in emotion:
        print(emotion_dict[i])
        print(softmax[emotion[id]].numpy()*100,"%")
        id+=1
    return

### sentences from songs to test with

In [4]:
unclear2 = "'Cause you see people, people, people, people Don't really know you (They don't really know you) They don't really know you 'Cause you see people, people, people They don't really know you (Mmm) They don't really know—"
unclear = "I've been drinking more alcohol for the past five days Did you check on me? Now, did you look for me? I walked in the room, eyes are red and I don't smoke banga Did you check on me? (Did you check on me?) Now, did you notice me?"

In [23]:
braggingLyrics = "She just hit my phone, she said, 'Tecca, you a winner' (Yeah) Took her home, then I turn her to a sinner, yeah I just got a check, I ate lobster for dinner She like my style, she tryna turn me to her nigga, yeah I just woke up, now they say that I’m a star I just wanna ball with my guys Drive off the Wock', he just caught a DUI He think he know the answers, nigga, like he Bill Nye"

In [21]:
sadMetaphoricLyrics = "Hello darkness, my old friend, I've come to talk with you again, because a vision softly creeping, left its seeds while I was sleeping."

In [27]:
goEasyOnMeSad = "There ain't no gold in this river That I've been washin' my hands in forever I know there is hope in these waters But I can't bring myself to swim When I am drowning in this silence Baby, let me in. Go easy on me, baby I was still a child Didn't get the chance to Feel the world around me I had no time to choose what I chose to do So go easy on me"

### Results

In [89]:
softmaxtestUnclear=getSoftmax(model, unclear, 3)

curiosity
99.90363121032715 %
neutral
0.034640790545381606 %
confusion
0.019302892906125635 %


In [88]:
softmaxtestUnclear2=getSoftmax(model,unclear2, 3)

neutral
99.9980092048645 %
annoyance
0.00058388282013766 %
disappointment
0.0003420888106120401 %


In [87]:
softmaxtestBraggingLyrics=getSoftmax(model,braggingLyrics, 3)

neutral
75.65684914588928 %
joy
16.482388973236084 %
excitement
4.176769405603409 %


In [91]:
softmaxtestsadMetaphoricLyrics=getSoftmax(model,sadMetaphoricLyrics, 3)

neutral
99.99864101409912 %
annoyance
0.00020481845695030643 %
caring
0.00011883536217283108 %


In [82]:
softmaxtestgoEasyOnMeSad=getSoftmax(model,goEasyOnMeSad, 3)

disappointment
63.13481330871582 %
realization
14.627596735954285 %
sadness
14.258602261543274 %


## SoftMax Recap:

1. There seems to be a dominance of neutral while the trailing classes seem to be more relevant. This does make sense considering the fact that not every lyric is emotionally charged.  

- However, this does raise the question, "How do we handle neutral?"

- similarly, "Do we want neutral as a resulting category?

2. Songs that are reliant on metaphores or to create emotion will not be captured via sentiment analysis

- In these cases, we would want the linear model to give results
- perhaps if this is specifially reated neutral we should rely on the linear classificiaton model

## 4. Conversion Without Special Neutral Considerations

Here we will:
1. scale emotions to (valence, energy) based on the valence-arousal mood wheel
2. multiply that value by the percent
3. add up x values as well as y values
4. place resulting values into the 8 main mood categories to get a class
5. if close to 0, label it neutral instead
    - If neutral, perhaps use as indicator to rely on 2nd model for classification

NOTE: Some words did not have a perfect point plotted, but there were some synonyms
- approval was mapped with satisfied
- caring was with passionate
- confused with a lessened amount of frustrated (1/2)
- curious was mapped with interested

### Mood setup verification (skip)

In [56]:
for key in range(len(emotion_dict)):
    print(emotion_dict[key])

admiration
amusement
anger
annoyance
approval
caring
confusion
curiosity
desire
disappointment
disapproval
disgust
embarrassment
excitement
fear
gratitude
grief
joy
love
nervousness
optimism
pride
realization
relief
remorse
sadness
surprise
neutral


In [61]:
# make a dictionary mapping all emotions to valence and arousal values
# valence is the x axis, arousal is the y axis of the circumplex model
emotionsAsValenceArousal= { 'admiration':(.6,.4),'amusement':(.6,.2),'anger':(-.8,.6),'annoyance':(-.6,.6),'approval':(.8,.6),'caring':(.6,-.2),'confusion':(-.2,.2),'curiosity':(0,.4),'desire':(.6,.6),'despair':(-.8,-.6),'disappointment':(-.6,-.6),'disapproval':(-.8,.65),'disgust':(-.8,.2),'embarrassment':(-.6,.4),'envy':(-.6,.4),'excitement':(.6,.8),'fear':(-.6,.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'joy':(.8,.2),'love':(.8,.4),'nervousness':(-.4,.6),'optimism':(.6,.2),'pride':(.6,.1),'realization':(.2,.2),'relief':(.4,-.4),'remorse':(-.6,-.4),'sadness':(-.8,-.2),'surprise':(.2,.6),'neutral':(0,0)}


In [65]:
emotion_dict = model.config.id2label
for i in range(len(emotion_dict)):
    print(emotionsAsValenceArousal[emotion_dict[i]])

(0.6, 0.4)
(0.6, 0.2)
(-0.8, 0.6)
(-0.6, 0.6)
(0.8, 0.6)
(0.6, -0.2)
(-0.2, 0.2)
(0, 0.4)
(0.6, 0.6)
(-0.6, -0.6)
(-0.8, 0.65)
(-0.8, 0.2)
(-0.6, 0.4)
(0.6, 0.8)
(-0.6, 0.8)
(0.6, -0.6)
(-0.6, -0.8)
(0.8, 0.2)
(0.8, 0.4)
(-0.4, 0.6)
(0.6, 0.2)
(0.6, 0.1)
(0.2, 0.2)
(0.4, -0.4)
(-0.6, -0.4)
(-0.8, -0.2)
(0.2, 0.6)
(0, 0)


### Function

In [ ]:
emotionsAsValenceArousal= { 'admiration':(.6,.4),'amusement':(.6,.2),'anger':(-.8,.6),'annoyance':(-.6,.6),'approval':(.8,.6),'caring':(.6,-.2),'confusion':(-.2,.2),'curiosity':(0,.4),'desire':(.6,.6),'despair':(-.8,-.6),'disappointment':(-.6,-.6),'disapproval':(-.8,.65),'disgust':(-.8,.2),'embarrassment':(-.6,.4),'envy':(-.6,.4),'excitement':(.6,.8),'fear':(-.6,.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'joy':(.8,.2),'love':(.8,.4),'nervousness':(-.4,.6),'optimism':(.6,.2),'pride':(.6,.1),'realization':(.2,.2),'relief':(.4,-.4),'remorse':(-.6,-.4),'sadness':(-.8,-.2),'surprise':(.2,.6),'neutral':(0,0)}


In [80]:
def convertScoresToLabels(softmaxScores, emotionsAsValenceArousal,emotion_dict):
    #convert the softmax scores to a valence and arousal score
    #softmax scores are in the order of the values in emotion_dict so we can use emotion[id] to get the softmax value
    id=0
    valence=0
    arousal=0
    for i in softmaxScores:
        valence+=i*emotionsAsValenceArousal[emotion_dict[id]][0]
        arousal+=i*emotionsAsValenceArousal[emotion_dict[id]][1]
        id+=1
    #convert valence and arousal from tensor to float
    valence = valence.numpy()
    arousal = arousal.numpy()
    return valence,arousal

### Results

In [93]:
convertScoresToLabels(softmaxtestUnclear, emotionsAsValenceArousal,emotion_dict)

(-0.00014550616, 0.39976904)

In [94]:
convertScoresToLabels(softmaxtestUnclear2, emotionsAsValenceArousal,emotion_dict)

(-6.28967e-06, 4.1340345e-06)

In [95]:
convertScoresToLabels(softmaxtestBraggingLyrics, emotionsAsValenceArousal,emotion_dict)

(0.17935042, 0.0792139)

In [96]:
convertScoresToLabels(softmaxtestsadMetaphoricLyrics, emotionsAsValenceArousal,emotion_dict)

(-1.578913e-06, 2.6779267e-06)

In [97]:
convertScoresToLabels(softmaxtestgoEasyOnMeSad, emotionsAsValenceArousal,emotion_dict)

(-0.4687149, -0.37509394)

## 5. Conversion With Special Neutral Considerations

## 6. Breaking up Lyrics to fit into model

In [126]:

longLyricsTest = """
Nah, nah, nah
Yeah, oh-oh
Yeah, oh no
We love you Tecca
Yeah, yeah
She just hit my phone, she said, "Tecca, you a winner" (That's Tec)
I just took her home, then I turn her to a sinner, yeah
Rambow

[Chorus]
She just hit my phone, she said, "Tecca, you a winner" (Yeah)
Took her home, then I turn her to a sinner, yeah
I just got a check, I ate lobster for dinner
She like my style, she tryna turn me to her nigga, yeah
I just woke up, now they say that I’m a star
I just wanna ball with my guys
Drive off the Wock', he just caught a DUI
He think he know the answers, nigga, like he Bill Nye
[Verse]
Bitch, I feel like A.I., nigga, never need to practice
I got two clips 'cause that chopper automatic
And I started talkin’ shit, I got tired of this rappin'
But I been making' hits, most these niggas, they been nappin'
And my lyrics hit her hard, like her nigga, he was stabbin'
LSD hit his brain, yeah, that nigga, he been tabbin'
I just got your shorty, yeah, my niggas, we been tappin'
Walk to the bank, Chief Keef, I be laughin'
Red, blue, yellow, nigga, you could pick your Power Ranger
I could fuck, but I won’t date ya
Getting bands, but I can’t save her
Fast nigga, but I can't chase her
360, quick scope, FaZe him
God-mode, aim-bot, laser
Cut you off, I got that razor

[Chorus]
She just hit my phone, she said, "Tecca, you a winner" (Yeah)
Took her home, then I turn her to a sinner, yeah
I just got a check, I ate lobster for dinner
She like my style, she tryna turn me to her nigga, yeah
I just woke up, now they sayin’ I'm a star
I just wanna ball with my guys
Drive off the Wock', he just caught a DUI
He think he got the answers, nigga, like he Bill Nye but I try not to die
"""

In [127]:
tokens = tokenizer.encode(longLyricsTest, return_tensors="pt")
tokens = tokens.cuda()
max_length = model.config.max_position_embeddings
#if the tokens are too long, break them up and put them in a list
if len(tokens[0]) > max_length:
    print('??')
    tokens = tokens.split(max_length)
#len(tokens[0][0])

#result = model(tokens)


??


## 7. Final Function For Inferences based on Lyrics

In [ ]:
def getMoodLabelFromLyrics(model, lyrics, printRawScores=False, printTopN=False):
    #probably first need to pass through to the functions the various dictionaries like emotion_dict and emotionsAsValenceArousal


    #Deterime the lyrics breakdown (depending on how this is done, my for loop will either be chunks of tokens or lyrics)

    #for lyricsChunk in LyricsChunks:
        #convert the lyrics to tokens
        #feed tokens to model and Get the softmax
        #apply the softmax values to the valence and arousal values
        #get the average valence and arousal values
        #get the label of the appropriate range and keep track of it

    #return the most common label